In [1]:
import ipyparallel as ipp
# for folders one level below the heirarchy
import sys
import os 
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
sys.path.append(parent_dir)

from aodfunctions.dependencies import *
from aodfunctions.general import *
from aodfunctions.settings import *
from aodfunctions.testbed import *
from aodfunctions.outofplane import *

# from aodfunctions.outofplane import *
from scipy.ndimage import zoom

cupyon=True

try: 
    import cupy as cp
    import cupyx.scipy.fft as cpfft
    import cupyx.scipy.ndimage
    from cupyx.scipy.ndimage import gaussian_filter1d as cp_gaussian_filter1d
    from cupyx.scipy.ndimage import gaussian_filter as cp_gaussian_filter
    from cupyx.scipy.ndimage import affine_transform as cp_affine_transform
    from cupy.lib.stride_tricks import as_strided

except ImportError:
    cp = np
    cpfft = spfft
    cp_gaussian_filter1d = sp_gaussian_filter1d
    cp_gaussian_filter = sp_gaussian_filter
    cp_affine_transform = sp_affine_transform
    cupyon = False
    from numpy.lib.stride_tricks import as_strided
    print("cupy not installed. Using numpy.")


# Set default plotting style
plt.style.use('seaborn-whitegrid')

# Set rcParams for scientific plotting
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({
    'figure.figsize': (10, 6),      # Default figure size
    'axes.titlesize': 16,           # Title font size
    'axes.labelsize': 14,           # Axes labels font size
    'lines.linewidth': 2,           # Line width
    'lines.markersize': 6,          # Marker size
    'xtick.labelsize': 12,          # X-tick labels font size
    'ytick.labelsize': 12,          # Y-tick labels font size
    'legend.fontsize': 12,          # Legend font size
    'font.family': 'serif',         # Font family
    'font.serif': ['Times New Roman'],
    'grid.alpha': 0.7,              # Grid transparency
    'grid.linestyle': '--',         # Grid line style
    'axes.grid': True,              # Show grid
    'savefig.dpi': 300,             # Save figure resolution
    'savefig.format': 'png'         # Default save format
})

import diffractsim
diffractsim.set_backend("CUDA") #Change the string to "CUDA" to use GPU acceleration

from diffractsim import MonochromaticField, Lens, nm, mm, cm, RectangularSlit


Cycletime in nanoseconds:  11538.461538461537
Aod pixel pitch in microns:  14.999999999999998
Fourier space spacing in microns:  0.09331054687500001
Fourier space aperture in microns:  382.20000000000005
Timestep in nanoseconds: 23.076923076923077


In [2]:
TRAP_DEPTH_MHz = 1e6 * 6.63e-34 #J for 1MHz trap
RB_MASS = 1.419E-25 #kg


## Setting constants
## time units are in us
## length units are in um
## mass units are in Rb mass
## trap depth/energy in units of Rb mass*(m/s)^2

F = 9000 #um This is almost the imaged AOD aperture diameter = 10,000 um
V = 650 * 10/7.5 #um/us The factor of 10/7.5 is from the imaging system from AOD to objective
M = 1
E0 = 40 * TRAP_DEPTH_MHz / RB_MASS #ideal trap depth
NA = 0.5
W = 1/np.pi/NA*0.8 #um
ZR = W/NA
LAMBDA = 0.8 #um
K = 2*np.pi/LAMBDA
AODaperture = 10000
N_pixel = 100
T_delay = AODaperture/V

XR=2500*1.5
rad_y = forward_focal_waist(XR,F,LAMBDA)

In [ ]:

def fourier_3D_potential(atom_loc,ideal_depth, Ac_list, AODaperture, XR, rad_y0):
    assert len(Ac_list) == N_pixel
    [x,y,z] = atom_loc
    # XR = backtract_beam_rad(rad_x0,F,LAMBDA)
    X_list = np.linspace(-AODaperture/2,AODaperture/2,N_pixel)
    A0_list = np.exp(-X_list**2/XR**2)
    A0_list /= sum(A0_list)
    temp1 = 1j* K* np.outer(X_list**2, z) /(2*F**2)
    temp2 = 1j*K*np.outer(X_list, x)/F
    temp3 = 1j*K*np.outer(Ac_list,(x+1.234321)/(x+1.234321))*V/F
    x_intensity_factor = abs(A0_list @ np.exp(temp1+temp2+temp3))**2 
    # x_intensity_factor = abs(sum(np.exp(temp1+temp2+temp3)))**2 / N_pixel**2
    rayleigh_y = np.pi*rad_y0**2/LAMBDA
    rad_y = rad_y0 * np.sqrt(1+(z/rayleigh_y)**2)
    y_intensity_factor = np.exp(-2*y**2/rad_y**2)*rad_y0/(rad_y)
    normalized_intensity = x_intensity_factor * y_intensity_factor
    return -ideal_depth * normalized_intensity

def fourier_3D_force_simp_and_slow(atom_loc,ideal_depth, Ac_list, AODaperture, XR, rad_y0):
    assert len(Ac_list) == N_pixel
    dx = 0.01
    atom_loc_dx = np.copy(atom_loc)
    atom_loc_dx[0] += dx
    atom_loc_dy = np.copy(atom_loc)
    atom_loc_dy[1] += dx
    atom_loc_dz = np.copy(atom_loc)
    atom_loc_dz[2] += dx
    Fx = (fourier_3D_potential(atom_loc,ideal_depth, Ac_list, AODaperture, XR, rad_y0)\
          - fourier_3D_potential(atom_loc_dx,ideal_depth, Ac_list, AODaperture, XR, rad_y0))/dx
    Fy = (fourier_3D_potential(atom_loc,ideal_depth, Ac_list, AODaperture, XR, rad_y0)\
          - fourier_3D_potential(atom_loc_dy,ideal_depth, Ac_list, AODaperture, XR, rad_y0))/dx
    Fz = (fourier_3D_potential(atom_loc,ideal_depth, Ac_list, AODaperture, XR, rad_y0)\
          - fourier_3D_potential(atom_loc_dz,ideal_depth, Ac_list, AODaperture, XR, rad_y0))/dx
    return np.array([Fx,Fy,Fz])

def fourier_3D_force(atom_loc,ideal_depth, Ac_list, AODaperture, XR, rad_y0):
    assert len(Ac_list) == N_pixel
    [x,y,z] = atom_loc
    # XR = backtract_beam_rad(rad_x0,F,LAMBDA)
    X_list = np.linspace(-AODaperture/2,AODaperture/2,N_pixel)
    A0_list = np.exp(-X_list**2/XR**2)
    A0_list /= sum(A0_list)
    rayleigh_y = np.pi*rad_y0**2/LAMBDA
    rad_y = rad_y0 * np.sqrt(1+(z/rayleigh_y)**2)
    y_intensity_factor = np.exp(-2*y**2/rad_y**2)*rad_y0/(rad_y)

    temp1 = 1j* K* np.outer(X_list**2, z) /(2*F**2)
    temp2 = 1j*K*np.outer(X_list, x)/F
    # temp3 = 1j*K*np.outer(Ac_list,np.ones(len(x)))*V/F
    temp3 = 1j*K*np.outer(Ac_list,(x+1.234321)/(x+1.234321))*V/F
    fourier_factor = np.exp(temp1+temp2+temp3) # shape of (N_pixel, N_particle)
    x_amp_factor = A0_list @ fourier_factor # shape of ( N_particle)
    x_intensity_factor = abs(x_amp_factor)**2  # shape of ( N_particle)
    normalized_intensity = x_intensity_factor * y_intensity_factor # shape of ( N_particle)
    
    x_grad_factor = np.conj(1j*K/F * np.einsum('i,i,ij->j', A0_list, X_list, fourier_factor))   # shape of ( N_particle)
    normalized_force_x = 2* np.real(x_amp_factor*x_grad_factor) * y_intensity_factor  # shape of ( N_particle)

    normalized_force_y = -normalized_intensity * (4*y/rad_y**2) # shape of ( N_particle)

    temp_A = z/(rayleigh_y**2+z**2)*(1-4*y**2/rad_y**2) 
    z_grad_factor = np.conj(1j*K/F**2/2 * np.einsum('i,i,ij->j', A0_list, X_list**2, fourier_factor))  
    normalized_force_z = -normalized_intensity * temp_A + y_intensity_factor * 2 * np.real(x_amp_factor*z_grad_factor)

    return ideal_depth * np.array([normalized_force_x,normalized_force_y,normalized_force_z])




fourier_force = fourier_3D_force
fourier_potential = fourier_3D_potential

In [ ]:

def kinetic_energy(phase_vec):
    v_vec = phase_vec[-3:]
    return (M/2) * np.sum(v_vec**2, axis=0)

def kinetic_energy_movframe(phase_vec,vc_vec):
    v_vec = phase_vec[-3:]
    return (M/2) * np.sum((v_vec-vc_vec)**2, axis=0)


def potential_energy(t, phase_vec,coeff):
    r_vec = phase_vec[:3]
    Ac_list = Ac_func(np.linspace(t,t-T_delay,N_pixel),coeff)
    return fourier_potential(r_vec,E0,Ac_list, AODaperture, XR, rad_y)

def potential_energy_t0(phase_vec):
    r_vec = phase_vec[:3]
    Ac_list = np.zeros(N_pixel)
    # Ac_list = Ac_func(np.linspace(0,0-T_delay,N_pixel))
    return fourier_potential(r_vec, E0, Ac_list, AODaperture, XR, rad_y)

def total_energy_movframe(t, phase_vec,coeff):
    arr = dxc_func(t,coeff)
    # arr = dxc_func(t)
    try:
        vc_vec = np.vstack((arr, np.zeros((2, arr.shape[0]))))
    except:
        vc_vec = np.array([arr,0,0])
    # print(f"kinetic shape = {kinetic_energy_movframe(phase_vec, vc_vec)[0].shape}")
    # print(f"potential shape = {potential_energy(t, phase_vec, coeff)[0].shape}")
    return kinetic_energy_movframe(phase_vec, vc_vec) + potential_energy(t, phase_vec, coeff)

def total_energy(t, phase_vec,coeff):
    return kinetic_energy(phase_vec) + potential_energy(t, phase_vec,coeff)

def total_energy_t0(phase_vec):
    return kinetic_energy(phase_vec) + potential_energy_t0(phase_vec)

########################## for sinesq ramp base ###########################
def xc_func_base(t):
    x = t/T_range
    return np.where(x <= 0, 0,            # If x <= 0, return 0
           np.where(x < 1, (1-np.cos(x*np.pi))/2 * L_range,         # If 0 < x < 1, return middle part
                    L_range))                # If x >= 1, return end part
def dxc_func_base(t):
    x = t/T_range
    return np.where(x <= 0, 0,            # If x <= 0, return 0
           np.where(x < 1, np.pi/2*np.sin(x*np.pi) * (L_range/T_range),         # If 0 < x < 1, return middle part
                    0))                # If x >= 1, return end part
def Ac_func_base(t):
    x = t/T_range
    return np.where(x <= 0, 0,            # If x <= 0, return 0
           np.where(x < 1, (x/2 - np.sin(np.pi*x)/(2*np.pi)) * T_range*L_range,         # If 0 < x < 1, return middle part
                    (x - 0.5) * T_range*L_range))                # If x >= 1, return end part


########################## for expansion perturbation using Bernstein Polynomials #########################
def xc_expand(t,n,HIGHEST_ORDER):
    x = t/T_range
    n_split = HIGHEST_ORDER+1
    return Bernstein_Poly(x, n_split, n) * L_range
def Ac_expand(t,n,HIGHEST_ORDER):
    x = t/T_range
    n_split = HIGHEST_ORDER+1
    return Int_Bernstein_Poly(x, n_split, n) * (L_range*T_range)
def dxc_expand(t,n,HIGHEST_ORDER):
    x = t/T_range
    n_split = HIGHEST_ORDER+1
    return dBernstein_Poly(x, n_split, n) * (L_range/T_range)

########################## for expansion perturbation using Bernstein Polynomials (not truncated triangle) #########################
def xc_expand(t,n,HIGHEST_DEGREE):
    x = t/T_range
    row, col = linear_to_triangle(n)
    d = row 
    i = col 
    # print([d,i])
    assert d<= HIGHEST_DEGREE
    return Bernstein_Poly(x, d, i) * L_range
def Ac_expand(t,n,HIGHEST_DEGREE):
    x = t/T_range
    row, col = linear_to_triangle(n)
    d = row 
    i = col 
    assert d<= HIGHEST_DEGREE
    return Int_Bernstein_Poly(x, d, i) * (L_range*T_range)
def dxc_expand(t,n,HIGHEST_DEGREE):
    x = t/T_range
    row, col = linear_to_triangle(n)
    d = row 
    i = col 
    assert d<= HIGHEST_DEGREE
    return dBernstein_Poly(x, d, i) * (L_range/T_range)
########################## for expansion perturbation using Bernstein Polynomials (truncated triangle) #########################

######################################################################################################################

def Ac_func(t,coeff):
    # assert len(coeff) == HIGHEST_ORDER
    HIGHEST_ORDER = len(coeff)
    HIGHEST_DEGREE = int((-1 + np.sqrt(1 + 8 * HIGHEST_ORDER)) / 2) - 1
    assert (HIGHEST_DEGREE+1)*(HIGHEST_DEGREE+2)/2 == HIGHEST_ORDER
    func_array = np.array([Ac_expand(t,n,HIGHEST_DEGREE) for n in range(0, HIGHEST_ORDER)])
    return coeff @ func_array +  Ac_func_base(t)
def xc_func(t,coeff):
    # assert len(coeff) == HIGHEST_ORDER
    HIGHEST_ORDER = len(coeff)
    HIGHEST_DEGREE = int((-1 + np.sqrt(1 + 8 * HIGHEST_ORDER)) / 2) - 1
    assert (HIGHEST_DEGREE+1)*(HIGHEST_DEGREE+2)/2 == HIGHEST_ORDER
    func_array = np.array([xc_expand(t,n,HIGHEST_DEGREE) for n in range(0, HIGHEST_ORDER)])
    return coeff @ func_array +  xc_func_base(t)
def dxc_func(t,coeff):
    # assert len(coeff) == HIGHEST_ORDER
    HIGHEST_ORDER = len(coeff)
    HIGHEST_DEGREE = int((-1 + np.sqrt(1 + 8 * HIGHEST_ORDER)) / 2) - 1
    assert (HIGHEST_DEGREE+1)*(HIGHEST_DEGREE+2)/2 == HIGHEST_ORDER
    func_array = np.array([dxc_expand(t,n,HIGHEST_DEGREE) for n in range(0, HIGHEST_ORDER)])
    return coeff @ func_array + dxc_func_base(t)

fourier_force = fourier_3D_force
fourier_potential = fourier_3D_potential

def time_derivative(t, phase_vec, coeff):
    # phase_vec = [x,y,z,vx,vy,vz]
    # potential_force_func(t, atom_loc)
    # fictitious_force_func(t)
    if len(phase_vec.shape) == 1 and phase_vec.shape == 6:
        r_vec = phase_vec[:3]
        v_vec = phase_vec[-3:]
        Ac_list = Ac_func(np.linspace(t,t-T_delay,N_pixel),coeff)
        potential_force = fourier_force(r_vec, E0, Ac_list, AODaperture, XR, rad_y)
        a_vec = (1/M) * potential_force
        return np.concatenate([v_vec,a_vec])
    elif len(phase_vec.shape) == 2 and phase_vec.shape[0] == 6:
        k = phase_vec.shape[-1]
        r_vec = phase_vec[:3]
        v_vec = phase_vec[-3:]
        Ac_list = Ac_func(np.linspace(t,t-T_delay,N_pixel),coeff)
        potential_force = fourier_force(r_vec, E0, Ac_list, AODaperture, XR, rad_y)
        a_vec = (1/M) * potential_force
        return np.concatenate([v_vec,a_vec])
    elif phase_vec.shape[0] % 6 == 0:
        # print("vectorized")
        k = phase_vec.shape[0] // 6
        r_vec = phase_vec[0:3*k].reshape(3,k)
        v_vec = phase_vec[-3*k:].reshape(3,k)
        Ac_list = Ac_func(np.linspace(t,t-T_delay,N_pixel),coeff)
        potential_force = fourier_force(r_vec, E0, Ac_list, AODaperture, XR, rad_y)
        a_vec = (1/M) * potential_force
        return np.concatenate([v_vec.ravel(),a_vec.ravel()])
    else:
        print(f"phase_vec shape= {phase_vec.shape} incorrect!")
        return 

In [5]:
import cupy as cp
import numpy as np


import cupy as cp

def create_linear_ramp_field(Nx, Ny):
    # Create a linear ramp from 0 to 1 in both the x and y directions
    x = cp.linspace(0, 1, Nx)
    y = cp.linspace(0, 1, Ny)
    X, Y = cp.meshgrid(x, y)

    # Create a field with intensity and phase ramps
    intensity_ramp = X * Y
    phase_ramp = 2 * cp.pi * X * Y

    # Combine intensity and phase to form the complex field
    field = intensity_ramp * cp.exp(1j * phase_ramp)
    
    return field


def compute_wave_propagation_at_point(field, wavelength, z, x_target, y_target):
    # Constants
    k = 2 * np.pi / wavelength  # wave number
    lambda_z = wavelength * z

    # Get the shape of the input field
    Ny, Nx = field.shape

    # Define the coordinate grid for the input field
    x = cp.linspace(-Nx//2, Nx//2, Nx)
    y = cp.linspace(-Ny//2, Ny//2, Ny)
    X, Y = cp.meshgrid(x, y)

    # Calculate the phase term inside the integral
    phase_term = cp.exp(1j * (k / (2 * z)) * ((x_target - X)**2 + (y_target - Y)**2))

    # Calculate the integral (sum) to get the value at the target point
    field_value_at_point = cp.sum(field * phase_term) / (1j * lambda_z)

    return field_value_at_point

# Example usage:
# Define the parameters
wavelength = 0.5e-6  # 500 nm
z = 0.2  # 20 cm propagation distance
x_target = 1e-3  # 1 mm
y_target = 2e-3  # 2 mm

# Example usage:
Nx, Ny = 1024, 1024
field = create_linear_ramp_field(Nx, Ny)
# Compute the field at the target point
field_value_at_point = compute_wave_propagation_at_point(field, wavelength, z, x_target, y_target)
print(f"Field value at point ({x_target}, {y_target}, {z}): {field_value_at_point}")


Field value at point (0.001, 0.002, 0.2): (2228741839.3727994-2151419648.1314816j)


In [6]:
def compute_fourier_value_at_point(field, x_target, y_target, dx, dy):
    # Compute the Fourier transform of the field
    field_ft = cp.fft.fftshift(cp.fft.fft2(cp.fft.ifftshift(field)))

    # Define the frequency grid
    Nx, Ny = field.shape
    fx = cp.fft.fftfreq(Nx, d=dx)
    fy = cp.fft.fftfreq(Ny, d=dy)
    FX, FY = cp.meshgrid(fx, fy)
    
    # Convert the target coordinates (x_target, y_target) to frequency space
    kx_target = 2 * cp.pi * x_target
    ky_target = 2 * cp.pi * y_target
    
    # Find the closest frequency grid point to the target
    idx_x = cp.argmin(cp.abs(FX[0] - kx_target))
    idx_y = cp.argmin(cp.abs(FY[:, 0] - ky_target))
    
    # Get the Fourier transform value at the target point
    fourier_value_at_point = field_ft[idx_y, idx_x]
    
    return fourier_value_at_point

# Example usage:
dx = 1.0 / Nx  # Assume the spatial resolution is the inverse of the grid size
dy = 1.0 / Ny

fourier_value_at_point = compute_fourier_value_at_point(field, 1e-3, 2e-3, dx, dy)
print(f"Fourier value at target point (kx, ky): {fourier_value_at_point}")


Fourier value at target point (kx, ky): (0.25048664433956175+0.0015366108426562164j)


In [7]:
# Parameters for wave propagation
wavelength = 0.5e-6  # 500 nm
z = 0.2  # 20 cm propagation distance
x_target = 1e-3  # 1 mm
y_target = 2e-3  # 2 mm

# Compute the field value at the target point using the direct propagation method
field_value_at_point_propagation = compute_wave_propagation_at_point(field, wavelength, z, x_target, y_target)

# Compute the Fourier value at the target point
fourier_value_at_point = compute_fourier_value_at_point(field, x_target, y_target, dx, dy)

print(f"Field value at target point (x, y, z) via propagation: {field_value_at_point_propagation}")
print(f"Fourier value at target point (kx, ky): {fourier_value_at_point}")


Field value at target point (x, y, z) via propagation: (2228741839.3727994-2151419648.1314816j)
Fourier value at target point (kx, ky): (0.25048664433956175+0.0015366108426562164j)
